<a href="https://colab.research.google.com/github/hamletbatista/seonderground/blob/master/SEOUnderground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Creando una Sección de FAQs y si Marcado Estructurado en 30 Minutos

##Agenda

Vamos a ver aplicaciones prácticas de generación de texto para SEO:

1. Preparación de entorno con Google Colab
2. Sacando texto para sumarizar y traducir
3. Usando sumarización con T5 y BART
4. Usando traducciones con T5
5. Traduciendo con Google Translate
6. Casos problemáticos en Español: Generación de Preguntas y  Respuestas Frequentes
7. Generación de Marcado Estructurado
7. Un poco de teoría de cómo todo esto funciona tan bien
8. Donde aprender más


## Preparación de entorno con Google Colab

In [1]:
%%capture
!pip install requests-html transformers jinja2

##Sacando texto para sumarizar y traducir

Puedes cambiar la dirección de la pagina en el formulario debajo

In [2]:
url = "https://www.elespectador.com/entretenimiento/gente/elaine-palacio-de-puerto-boyaca-a-las-pasarelas-de-nueva-york/" #@param {type:"string"}
selector = "p" #@param {type:"string"}



In [3]:
# Sacado de https://www.searchenginejournal.com/generate-quality-faqs-faqpage-schemas-with-python/380004/

from requests_html import HTMLSession
sesión = HTMLSession()

with sesión.get(url) as r:

    párrafo = r.html.find(selector, first=False)

    texto = " ".join([ p.text for p in párrafo])


Veamos el texto que extraímos

In [4]:
texto

" La modelo colombiana de 18 años apareció en la edición especial de septiembre de la revista “Vogue” Italia. Tiene contratos con marcas como Victoria’s Secret, Reebok y Burberry. “Mi reinita, te lo dije. Tienes 18 años y mira dónde estás”, le dijo María Flor Rentería a su nieta Elaine Palacio Mosquera cuando la vio en la portada de Vogue Italia. Ella la impulsó para que se metiera de modelo. La abuela cree que su nieta es la Naomi Campbell colombiana. “Fue quien me apoyó pagando las mensualidades en la academia de modelaje. Tuve que aprender sobre qué era ser una modelo”, cuenta. Elaine Palacio fue elegida como una de los cien personajes que aparecieron en la portada de la revista Vogue Italia en la edición especial de septiembre. La publicación decidió hacer un centenar de portadas diferentes para ese mes, el más importante del año, porque las editoriales reciben pautas de grandes empresas para mostrar las tendencias de otoño-invierno. “Vogue” escogió a famosas como Cindy Crawford, B

Se ve bien! 🤓

##Usando sumarización con BART

In [7]:
from transformers import pipeline

In [ ]:
# USando bart con pytorch
sumarizador_bart = pipeline("summarization")

In [10]:
resumen = sumarizador_bart(texto, min_length=50, max_length=250)

Token indices sequence length is longer than the specified maximum sequence length for this model (3175 > 1024). Running this sequence through the model will result in indexing errors


IndexError: ignored

Vamos a truncar el texto para que funcione

In [11]:
resumen = sumarizador_bart(texto[:1024], min_length=50, max_length=250)

In [14]:
resumen[0]["summary_text"]

' Elaine Palacio, 18, apareció in edición especial de septiembre of Vogue Italia . Tiene contratos con marcas como Victoria’s Secret, Reebok y Burberry . “Vogue” escogió a famosas como Cindy Crawford, Bella Hadid y Emily Ratajk .'

##Usando sumarización con T5

In [16]:
sumarizador_t5 = pipeline("summarization", model="t5-base", tokenizer="t5-base")

resumen_base = sumarizador_t5(texto, min_length=50, max_length=250)

Token indices sequence length is longer than the specified maximum sequence length for this model (3764 > 512). Running this sequence through the model will result in indexing errors


In [17]:
resumen_base[0]["summary_text"]

"i'm black, I'm Afro Can't be more Honored to be part of this amazing story, thank you . #vogueitalia100 covers, 100 people, 100 stories, 1 Elaine ."

In [8]:
sumarizador_t5 = pipeline("summarization", model="t5-large", tokenizer="t5-large")

resumen_grande = sumarizador_t5(texto, min_length=50, max_length=250)

Token indices sequence length is longer than the specified maximum sequence length for this model (3764 > 512). Running this sequence through the model will result in indexing errors


In [9]:
resumen_grande[0]["summary_text"]

'Elaine Palacio Mosquera, de 18 aos, es una modelo afro-colombiana que elegió en la edición especial de septiembre de la revista “Vogue” Italia. “Me reinita, te lo dije”, dijo Mara Flor Rentera a su nieta Elaine palacio en Divina!'

## Haciendo Traducciones con T5

Vamos a usar este modelo que traduce al Portugues https://huggingface.co/mrm8488/t5-small-finetuned-translation-es-to-pt

In [19]:
#mrm8488/t5-small-finetuned-translation-es-to-pt

es_to_pt_traducción = pipeline("translation_en_to_de", model="mrm8488/t5-small-finetuned-translation-es-to-pt", 
                            tokenizer="mrm8488/t5-small-finetuned-translation-es-to-pt")


In [21]:
traducción = es_to_pt_traducción(texto, min_length=50, max_length=250)

Token indices sequence length is longer than the specified maximum sequence length for this model (3767 > 512). Running this sequence through the model will result in indexing errors
Your input_length: 3767 is bigger than 0.9 * max_length: 250. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


In [24]:
traducción[0]["translation_text"]

"'' a modelo de 18 anos especial da revista '' '' . têm contratos com marcas como a Victoria’s Secret , reebok e a burberry .  ''  '' eu ainda ,  '' eu ainda , eu aconteceu em vogueitalia . ''"

##Traduciendo con Google Translate

https://cloud.google.com/translate/docs/basic/quickstart

In [5]:
%%capture
!pip install googletrans

In [6]:
from googletrans import Translator

In [7]:
translator = Translator()

In [20]:
result = translator.translate(texto, dest='pt')


In [21]:
print(result.src)
print(result.dest)
print(result.origin)
print(result.text)
print(result.pronunciation)

es
pt
 La modelo colombiana de 18 años apareció en la edición especial de septiembre de la revista “Vogue” Italia. Tiene contratos con marcas como Victoria’s Secret, Reebok y Burberry. “Mi reinita, te lo dije. Tienes 18 años y mira dónde estás”, le dijo María Flor Rentería a su nieta Elaine Palacio Mosquera cuando la vio en la portada de Vogue Italia. Ella la impulsó para que se metiera de modelo. La abuela cree que su nieta es la Naomi Campbell colombiana. “Fue quien me apoyó pagando las mensualidades en la academia de modelaje. Tuve que aprender sobre qué era ser una modelo”, cuenta. Elaine Palacio fue elegida como una de los cien personajes que aparecieron en la portada de la revista Vogue Italia en la edición especial de septiembre. La publicación decidió hacer un centenar de portadas diferentes para ese mes, el más importante del año, porque las editoriales reciben pautas de grandes empresas para mostrar las tendencias de otoño-invierno. “Vogue” escogió a famosas como Cindy Crawfo

In [22]:
result = translator.translate(texto, dest="en")

In [23]:
print(result.src)
print(result.dest)
print(result.origin)
print(result.text)
print(result.pronunciation)

es
en
 La modelo colombiana de 18 años apareció en la edición especial de septiembre de la revista “Vogue” Italia. Tiene contratos con marcas como Victoria’s Secret, Reebok y Burberry. “Mi reinita, te lo dije. Tienes 18 años y mira dónde estás”, le dijo María Flor Rentería a su nieta Elaine Palacio Mosquera cuando la vio en la portada de Vogue Italia. Ella la impulsó para que se metiera de modelo. La abuela cree que su nieta es la Naomi Campbell colombiana. “Fue quien me apoyó pagando las mensualidades en la academia de modelaje. Tuve que aprender sobre qué era ser una modelo”, cuenta. Elaine Palacio fue elegida como una de los cien personajes que aparecieron en la portada de la revista Vogue Italia en la edición especial de septiembre. La publicación decidió hacer un centenar de portadas diferentes para ese mes, el más importante del año, porque las editoriales reciben pautas de grandes empresas para mostrar las tendencias de otoño-invierno. “Vogue” escogió a famosas como Cindy Crawfo

Vamos a generar FAQs con texto traducido

In [24]:
result.text

'The 18-year-old Colombian model appeared in the special September issue of “Vogue” Italia magazine. He has contracts with brands such as Victoria’s Secret, Reebok and Burberry. "My little warbler, I told you. You are 18 years old and look where you are, ”María Flor Renteria told her granddaughter Elaine Palacio Mosquera when she saw her on the cover of Vogue Italia. She encouraged her to become a model. The grandmother believes that her granddaughter is the Colombian Naomi Campbell. “He was the one who supported me by paying the monthly payments at the modeling academy. I had to learn what it was like to be a model, ”she says. Elaine Palacio was chosen as one of the 100 characters to appear on the cover of Vogue Italia magazine in the September special issue. The publication decided to do a hundred different covers for that month, the most important of the year, because publishers receive guidelines from large companies to show autumn-winter trends. “Vogue” chose celebrities such as C

In [13]:
!python -m nltk.downloader punkt
!git clone https://github.com/patil-suraj/question_generation.git

/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Cloning into 'question_generation'...
remote: Enumerating objects: 173, done.
remote: Total 173 (delta 0), reused 0 (delta 0), pack-reused 173
Receiving objects: 100% (173/173), 264.42 KiB | 848.00 KiB/s, done.
Resolving deltas: 100% (90/90), done.


In [14]:
%cd question_generation


/content/question_generation


## Corrección temporal

In [15]:
# temporary solution
#https://github.com/patil-suraj/question_generation/issues/22

%%writefile /content/question_generation/pipelines.py 

import itertools
import logging
from typing import Optional, Dict, Union

from nltk import sent_tokenize

import torch
from transformers import(
    AutoModelForSeq2SeqLM, 
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
)

logger = logging.getLogger(__name__)

class QGPipeline:
    """Poor man's QG pipeline"""
    def __init__(
        self,
        model: PreTrainedModel,
        tokenizer: PreTrainedTokenizer,
        ans_model: PreTrainedModel,
        ans_tokenizer: PreTrainedTokenizer,
        qg_format: str,
        use_cuda: bool
    ):
        self.model = model
        self.tokenizer = tokenizer

        self.ans_model = ans_model
        self.ans_tokenizer = ans_tokenizer

        self.qg_format = qg_format

        self.device = "cuda" if torch.cuda.is_available() and use_cuda else "cpu"
        self.model.to(self.device)

        if self.ans_model is not self.model:
            self.ans_model.to(self.device)

        assert self.model.__class__.__name__ in ["T5ForConditionalGeneration", "BartForConditionalGeneration"]
        
        if "T5ForConditionalGeneration" in self.model.__class__.__name__:
            self.model_type = "t5"
        else:
            self.model_type = "bart"

    def __call__(self, inputs: str):
        inputs = " ".join(inputs.split())
        sents, answers = self._extract_answers(inputs)
        flat_answers = list(itertools.chain(*answers))
        
        if len(flat_answers) == 0:
          return []

        if self.qg_format == "prepend":
            qg_examples = self._prepare_inputs_for_qg_from_answers_prepend(inputs, answers)
        else:
            qg_examples = self._prepare_inputs_for_qg_from_answers_hl(sents, answers)
        
        qg_inputs = [example['source_text'] for example in qg_examples]
        questions = self._generate_questions(qg_inputs)
        output = [{'answer': example['answer'], 'question': que} for example, que in zip(qg_examples, questions)]
        return output
    
    def _generate_questions(self, inputs):
        inputs = self._tokenize(inputs, padding=True, truncation=True)
        
        outs = self.model.generate(
            input_ids=inputs['input_ids'].to(self.device), 
            attention_mask=inputs['attention_mask'].to(self.device), 
            max_length=32,
            num_beams=4,
        )
        
        questions = [self.tokenizer.decode(ids, skip_special_tokens=True) for ids in outs]
        return questions
    
    def _extract_answers(self, context):
        sents, inputs = self._prepare_inputs_for_ans_extraction(context)
        inputs = self._tokenize(inputs, padding=True, truncation=True)

        outs = self.ans_model.generate(
            input_ids=inputs['input_ids'].to(self.device), 
            attention_mask=inputs['attention_mask'].to(self.device), 
            max_length=32,
        )
        
        dec = [self.ans_tokenizer.decode(ids, skip_special_tokens=False) for ids in outs]
        answers = [item.split('<sep>') for item in dec]
        answers = [i[:-1] for i in answers]
        
        return sents, answers
    
    def _tokenize(self,
        inputs,
        padding=True,
        truncation=True,
        add_special_tokens=True,
        max_length=512
    ):
        inputs = self.tokenizer.batch_encode_plus(
            inputs, 
            max_length=max_length,
            add_special_tokens=add_special_tokens,
            truncation=truncation,
            padding="max_length" if padding else False,
            pad_to_max_length=padding,
            return_tensors="pt"
        )
        return inputs
    
    def _prepare_inputs_for_ans_extraction(self, text):
        sents = sent_tokenize(text)

        inputs = []
        for i in range(len(sents)):
            source_text = "extract answers:"
            for j, sent in enumerate(sents):
                if i == j:
                    sent = "<hl> %s <hl>" % sent
                source_text = "%s %s" % (source_text, sent)
                source_text = source_text.strip()
            
            if self.model_type == "t5":
                source_text = source_text + " </s>"
            inputs.append(source_text)

        return sents, inputs
    
    def _prepare_inputs_for_qg_from_answers_hl(self, sents, answers):
        inputs = []
        for i, answer in enumerate(answers):
            if len(answer) == 0: continue
            sent = sents[i]
            for answer_text in answer:
                sents_copy = sents[:]
                
                answer_text = answer_text.strip()
                
                #ans_start_idx = sent.index(answer_text)
                #bypass missmatched Q/As. See https://github.com/patil-suraj/question_generation/issues/22
                if answer_text in sent: 
                  ans_start_idx = sent.index(answer_text) 
                else: 
                  continue
                
                sent = f"{sent[:ans_start_idx]} <hl> {answer_text} <hl> {sent[ans_start_idx + len(answer_text): ]}"
                sents_copy[i] = sent
                
                source_text = " ".join(sents_copy)
                source_text = f"generate question: {source_text}" 
                if self.model_type == "t5":
                    source_text = source_text + " </s>"
                
                inputs.append({"answer": answer_text, "source_text": source_text})
        
        return inputs
    
    def _prepare_inputs_for_qg_from_answers_prepend(self, context, answers):
        flat_answers = list(itertools.chain(*answers))
        examples = []
        for answer in flat_answers:
            source_text = f"answer: {answer} context: {context}"
            if self.model_type == "t5":
                source_text = source_text + " </s>"
            
            examples.append({"answer": answer, "source_text": source_text})
        return examples

    
class MultiTaskQAQGPipeline(QGPipeline):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    
    def __call__(self, inputs: Union[Dict, str]):
        if type(inputs) is str:
            # do qg
            return super().__call__(inputs)
        else:
            # do qa
            return self._extract_answer(inputs["question"], inputs["context"])
    
    def _prepare_inputs_for_qa(self, question, context):
        source_text = f"question: {question}  context: {context}"
        if self.model_type == "t5":
            source_text = source_text + " </s>"
        return  source_text
    
    def _extract_answer(self, question, context):
        source_text = self._prepare_inputs_for_qa(question, context)
        inputs = self._tokenize([source_text], padding=False)
    
        outs = self.model.generate(
            input_ids=inputs['input_ids'].to(self.device), 
            attention_mask=inputs['attention_mask'].to(self.device), 
            max_length=16,
        )

        answer = self.tokenizer.decode(outs[0], skip_special_tokens=True)
        return answer


class E2EQGPipeline:
    def __init__(
        self,
        model: PreTrainedModel,
        tokenizer: PreTrainedTokenizer,
        use_cuda: bool
    ) :

        self.model = model
        self.tokenizer = tokenizer

        self.device = "cuda" if torch.cuda.is_available() and use_cuda else "cpu"
        self.model.to(self.device)

        assert self.model.__class__.__name__ in ["T5ForConditionalGeneration", "BartForConditionalGeneration"]
        
        if "T5ForConditionalGeneration" in self.model.__class__.__name__:
            self.model_type = "t5"
        else:
            self.model_type = "bart"
        
        self.default_generate_kwargs = {
            "max_length": 256,
            "num_beams": 4,
            "length_penalty": 1.5,
            "no_repeat_ngram_size": 3,
            "early_stopping": True,
        }
    
    def __call__(self, context: str, **generate_kwargs):
        inputs = self._prepare_inputs_for_e2e_qg(context)

        # TODO: when overrding default_generate_kwargs all other arguments need to be passsed
        # find a better way to do this
        if not generate_kwargs:
            generate_kwargs = self.default_generate_kwargs
        
        input_length = inputs["input_ids"].shape[-1]
        
        # max_length = generate_kwargs.get("max_length", 256)
        # if input_length < max_length:
        #     logger.warning(
        #         "Your max_length is set to {}, but you input_length is only {}. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)".format(
        #             max_length, input_length
        #         )
        #     )

        outs = self.model.generate(
            input_ids=inputs['input_ids'].to(self.device), 
            attention_mask=inputs['attention_mask'].to(self.device),
            **generate_kwargs
        )

        prediction = self.tokenizer.decode(outs[0], skip_special_tokens=True)
        questions = prediction.split("<sep>")
        questions = [question.strip() for question in questions[:-1]]
        return questions
    
    def _prepare_inputs_for_e2e_qg(self, context):
        source_text = f"generate questions: {context}"
        if self.model_type == "t5":
            source_text = source_text + " </s>"
        
        inputs = self._tokenize([source_text], padding=False)
        return inputs
    
    def _tokenize(
        self,
        inputs,
        padding=True,
        truncation=True,
        add_special_tokens=True,
        max_length=512
    ):
        inputs = self.tokenizer.batch_encode_plus(
            inputs, 
            max_length=max_length,
            add_special_tokens=add_special_tokens,
            truncation=truncation,
            padding="max_length" if padding else False,
            pad_to_max_length=padding,
            return_tensors="pt"
        )
        return inputs


SUPPORTED_TASKS = {
    "question-generation": {
        "impl": QGPipeline,
        "default": {
            "model": "valhalla/t5-small-qg-hl",
            "ans_model": "valhalla/t5-small-qa-qg-hl",
        }
    },
    "multitask-qa-qg": {
        "impl": MultiTaskQAQGPipeline,
        "default": {
            "model": "valhalla/t5-small-qa-qg-hl",
        }
    },
    "e2e-qg": {
        "impl": E2EQGPipeline,
        "default": {
            "model": "valhalla/t5-small-e2e-qg",
        }
    }
}

def pipeline(
    task: str,
    model: Optional = None,
    tokenizer: Optional[Union[str, PreTrainedTokenizer]] = None,
    qg_format: Optional[str] = "highlight",
    ans_model: Optional = None,
    ans_tokenizer: Optional[Union[str, PreTrainedTokenizer]] = None,
    use_cuda: Optional[bool] = True,
    **kwargs,
):
    # Retrieve the task
    if task not in SUPPORTED_TASKS:
        raise KeyError("Unknown task {}, available tasks are {}".format(task, list(SUPPORTED_TASKS.keys())))

    targeted_task = SUPPORTED_TASKS[task]
    task_class = targeted_task["impl"]

    # Use default model/config/tokenizer for the task if no model is provided
    if model is None:
        model = targeted_task["default"]["model"]
    
    # Try to infer tokenizer from model or config name (if provided as str)
    if tokenizer is None:
        if isinstance(model, str):
            tokenizer = model
        else:
            # Impossible to guest what is the right tokenizer here
            raise Exception(
                "Impossible to guess which tokenizer to use. "
                "Please provided a PretrainedTokenizer class or a path/identifier to a pretrained tokenizer."
            )
    
    # Instantiate tokenizer if needed
    if isinstance(tokenizer, (str, tuple)):
        if isinstance(tokenizer, tuple):
            # For tuple we have (tokenizer name, {kwargs})
            tokenizer = AutoTokenizer.from_pretrained(tokenizer[0], **tokenizer[1])
        else:
            tokenizer = AutoTokenizer.from_pretrained(tokenizer)
    
    # Instantiate model if needed
    if isinstance(model, str):
        model = AutoModelForSeq2SeqLM.from_pretrained(model)
    
    if task == "question-generation":
        if ans_model is None:
            # load default ans model
            ans_model = targeted_task["default"]["ans_model"]
            ans_tokenizer = AutoTokenizer.from_pretrained(ans_model)
            ans_model = AutoModelForSeq2SeqLM.from_pretrained(ans_model)
        else:
            # Try to infer tokenizer from model or config name (if provided as str)
            if ans_tokenizer is None:
                if isinstance(ans_model, str):
                    ans_tokenizer = ans_model
                else:
                    # Impossible to guest what is the right tokenizer here
                    raise Exception(
                        "Impossible to guess which tokenizer to use. "
                        "Please provided a PretrainedTokenizer class or a path/identifier to a pretrained tokenizer."
                    )
            
            # Instantiate tokenizer if needed
            if isinstance(ans_tokenizer, (str, tuple)):
                if isinstance(ans_tokenizer, tuple):
                    # For tuple we have (tokenizer name, {kwargs})
                    ans_tokenizer = AutoTokenizer.from_pretrained(ans_tokenizer[0], **ans_tokenizer[1])
                else:
                    ans_tokenizer = AutoTokenizer.from_pretrained(ans_tokenizer)

            if isinstance(ans_model, str):
                ans_model = AutoModelForSeq2SeqLM.from_pretrained(ans_model)
    
    if task == "e2e-qg":
        return task_class(model=model, tokenizer=tokenizer, use_cuda=use_cuda)
    elif task == "question-generation":
        return task_class(model=model, tokenizer=tokenizer, ans_model=ans_model, ans_tokenizer=ans_tokenizer, qg_format=qg_format, use_cuda=use_cuda)
    else:
        return task_class(model=model, tokenizer=tokenizer, ans_model=model, ans_tokenizer=tokenizer, qg_format=qg_format, use_cuda=use_cuda)



Overwriting /content/question_generation/pipelines.py


## Generación de Preguntas y Respuetas Frecuentes

In [16]:
from pipelines import pipeline


In [17]:
nlp = pipeline("multitask-qa-qg")

In [25]:
faqs = nlp(result.text)

In [26]:
faqs

[{'answer': 'September',
  'question': 'When did the Colombian model appear on Vogue Italia magazine?'},
 {'answer': 'Victoria’s Secret, Reebok and Burberry',
  'question': 'What brands does the Colombian model have contracts with?'},
 {'answer': 'My little warbler',
  'question': 'What was the name of the model who appeared on the cover of Vogue Italia?'},
 {'answer': '18',
  'question': 'How old are you when she saw Elaine Palacio Mosquera on the cover of Vogue Italia?'},
 {'answer': 'Naomi Campbell',
  'question': 'Who is the grandmother of Elaine Palacio Mosquera?'},
 {'answer': 'paying the monthly payments at the modeling academy',
  'question': 'What did Naomi Campbell support her?'},
 {'answer': 'learn what it was like to be a model',
  'question': 'What did Naomi Campbell do when she saw Elaine Palacio Mosquera on the cover of Vogue Italia?'},
 {'answer': 'Elaine Palacio',
  'question': 'Who was chosen as one of the 100 characters to appear on the cover of Vogue Italia?'},
 {'a

Vamos a traducir de Ingles a Español

In [27]:
faqs_traducidas = list()

In [28]:
for faq in faqs:
  pregunta = translator.translate(faq["question"], dest="es")
  respuesta = translator.translate(faq["answer"], dest="es")

  faqs_traducidas.append({"pregunta": pregunta.text, "respuesta": respuesta.text})

In [29]:
faqs_traducidas


[{'pregunta': '¿Cuándo apareció la modelo colombiana en la revista Vogue Italia?',
  'respuesta': 'septiembre'},
 {'pregunta': '¿Con qué marcas tiene contratos el modelo colombiano?',
  'respuesta': "Victoria's Secret, Reebok y Burberry"},
 {'pregunta': '¿Cómo se llamaba la modelo que apareció en la portada de Vogue Italia?',
  'respuesta': 'Mi pequeña curruca'},
 {'pregunta': '¿Qué edad tienes cuando vio a Elaine Palacio Mosquera en la portada de Vogue Italia?',
  'respuesta': '18'},
 {'pregunta': '¿Quién es la abuela de Elaine Palacio Mosquera?',
  'respuesta': 'Naomi Campbell'},
 {'pregunta': '¿Qué la apoyó Naomi Campbell?',
  'respuesta': 'pagando los pagos mensuales en la academia de modelaje'},
 {'pregunta': '¿Qué hizo Naomi Campbell cuando vio a Elaine Palacio Mosquera en la portada de Vogue Italia?',
  'respuesta': 'aprender lo que era ser modelo'},
 {'pregunta': '¿Quién fue elegido como uno de los 100 personajes que aparecerán en la portada de Vogue Italia?',
  'respuesta': 'E

## Creando el Marcado Estructurado

Usamos una plantilla de Jinja2 para hacer la generación. Puedes aprender más aquí https://jinja.palletsprojects.com/en/2.11.x/


In [33]:
plantilla_faqpage="""<script type="application/ld+json">

{

"@context": "https://schema.org",

"@type": "FAQPage",

"mainEntity": [

{% for faq in faqs %}

{

"@type": "Question",

"name": {{faq.pregunta|tojson}},

"acceptedAnswer": {

"@type": "Answer",

"text": {{faq.respuesta|tojson}}

}

}{{ "," if not loop.last }}

{% endfor %}

]

}

</script>"""

In [35]:
from jinja2 import Template

plantilla=Template(plantilla_faqpage)

salida_faqpage=plantilla.render(faqs=faqs_traducidas)


Veamos el marcado que generamos

In [36]:
print(salida_faqpage)

<script type="application/ld+json">

{

"@context": "https://schema.org",

"@type": "FAQPage",

"mainEntity": [



{

"@type": "Question",

"name": "\u00bfCu\u00e1ndo apareci\u00f3 la modelo colombiana en la revista Vogue Italia?",

"acceptedAnswer": {

"@type": "Answer",

"text": "septiembre"

}

},



{

"@type": "Question",

"name": "\u00bfCon qu\u00e9 marcas tiene contratos el modelo colombiano?",

"acceptedAnswer": {

"@type": "Answer",

"text": "Victoria\u0027s Secret, Reebok y Burberry"

}

},



{

"@type": "Question",

"name": "\u00bfC\u00f3mo se llamaba la modelo que apareci\u00f3 en la portada de Vogue Italia?",

"acceptedAnswer": {

"@type": "Answer",

"text": "Mi peque\u00f1a curruca"

}

},



{

"@type": "Question",

"name": "\u00bfQu\u00e9 edad tienes cuando vio a Elaine Palacio Mosquera en la portada de Vogue Italia?",

"acceptedAnswer": {

"@type": "Answer",

"text": "18"

}

},



{

"@type": "Question",

"name": "\u00bfQui\u00e9n es la abuela de Elaine Palacio Mosqu

Lo podemos ver [aquí](https://gist.github.com/hamletbatista/5958be10747621faf03cee4dcd7b8124) 